In [1]:
import numpy as np
from urllib import request
import gzip
import pickle
import os
import torch
from torch.nn.utils.rnn import pad_sequence

In [2]:
from torch import LongTensor
from torch.autograd import Variable
from torch.nn import Embedding
from torch.nn.utils.rnn import pack_padded_sequence

In [3]:
import wget, os, gzip, pickle, random, re, sys

IMDB_URL = ## removed (internal)
IMDB_FILE = ## removed (internal)

PAD, START, END, UNK = '.pad', '.start', '.end', '.unk'

def load_imdb(final=False, val=5000, seed=0, voc=None, char=False):

    cst = 'char' if char else 'word'

    imdb_url = IMDB_URL.format(cst)
    imdb_file = IMDB_FILE.format(cst)

    if not os.path.exists(imdb_file):
        wget.download(imdb_url)

    with gzip.open(imdb_file) as file:
        sequences, labels, i2w, w2i = pickle.load(file)

    #print(len(w2i))
    #print(len(i2w))
    if voc is not None and voc < len(i2w):
        nw_sequences = {}

        i2w = i2w[:voc]
        w2i = {w: i for i, w in enumerate(i2w)}

        mx, unk = voc, w2i['.unk']
        for key, seqs in sequences.items():
            nw_sequences[key] = []
            for seq in seqs:
                seq = [s if s < mx else unk for s in seq]
                nw_sequences[key].append(seq)

        sequences = nw_sequences

    if final:
        return (sequences['train'], labels['train']), (sequences['test'], labels['test']), (i2w, w2i), 2

    # Make a validation split
    random.seed(seed)

    x_train, y_train = [], []
    x_val, y_val = [], []

    val_ind = set( random.sample(range(len(sequences['train'])), k=val) )
    for i, (s, l) in enumerate(zip(sequences['train'], labels['train'])):
        if i in val_ind:
            x_val.append(s)
            y_val.append(l)
        else:
            x_train.append(s)
            y_train.append(l)

    return (x_train, y_train), \
           (x_val, y_val), \
           (i2w, w2i), 2

In [9]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = load_imdb(final=False)

In [10]:
def collate_fn(batch):
    data_list = []
    for _data in batch:
        data_list.extend(_data)
       
    one_batch = torch.tensor(data_list, dtype = torch.long)
    return (one_batch)


In [11]:
from torch.utils.data import DataLoader

dataloader = DataLoader(x_train[:10], batch_size=2, shuffle=False, collate_fn = collate_fn)

In [12]:
#Padding on Sorted batches 
seq_lengths = LongTensor(list(map(len, dataloader)))
seq_tensor = torch.zeros((len(dataloader), seq_lengths.max())).long()

for idx, (seq, seqlen) in enumerate(zip(dataloader, seq_lengths)):
    seq_tensor[idx, :seqlen] = LongTensor(seq)

seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
seq_tensor = seq_tensor[perm_idx]

In [13]:
for row in seq_tensor:
    print(row)


tensor([   13,    90,    25,   123,   138,    13,    42,    14,    19,    40,
           73,    22,    13,   116,    79,  1414,     7,   153,    11,    84,
           19,   262,     4,   210, 52811,   598,    35,   240,    14,  2423,
         8932,    55,    24,    31,   418,   257,    15,   310,   285])
tensor([  14,    9,    4, 6750,   19,  320,    7, 3526, 3931, 1911,  167,   22,
          43,   29,   60,  986,  388,    6,  664,    7,  129,   27,  889,    8,
        2578,   89,  750, 2297,    5, 7684,   78,   14,   19,    9,    0,    0,
           0,    0,    0])
tensor([   60,   913,   366,    19,   118,   836,    44,   431,   902,    60,
          286,    35,    34,  1834,    11, 15069,   398,  1415,     9,     4,
          122,   398,     7, 15069,    11,    16,    62,   516,   398,     7,
        15069,     0,     0,     0,     0,     0,     0,     0,     0])
tensor([10721,     4, 10956,   129,     6,   124, 88114,     5,     6,    19,
           93,  4118,   198,   351, 17697, 